Import Library

In [54]:
import pandas as pd
import json
from openai import OpenAI
import os

Test excel

In [2]:
df = pd.read_excel(r"C:\\Users\\btuud\\Downloads\\Component_generator\\train_langflow_custom_component_data.xlsx")
print(df)

    Id                 Name Difficulty                      Category  \
0    1   Sum of two numbers       Easy                          Math   
1    2                 Echo       Easy            Message Processing   
2    3           Chat Input     Medium               Chat Components   
3    4           Text Input       Easy            Message Processing   
4    5          Chat Output     Medium               Chat Components   
5    6          Text Output       Easy            Message Processing   
6    7   Sub of two numbers       Easy                          Math   
7    8  Sum of three number       Easy                          Math   
8    9                  URL       Hard                  Web Scraping   
9   10                 File     Medium              Document Loading   
10  11               Prompt       Easy                        Prompt   
11  12           Parse Data     Meduim               Data Processing   
12  13      Data to Message     Meduim               Data Proces

1. Prepare Python data

In [31]:
#Prepare python data to JSONL
def convert_python_one_line(python_code):
    jsonl_line = json.dumps(python_code, ensure_ascii=False)
    return jsonl_line

def convert_python_to_jsonl(python_code, description, num_input, input_type, num_output, output_type, output_file):
    
    prompt = f"Generate a LangFlow custom component in python code that {description}. "
    prompt += f"The component should have {num_input} {input_type} input"
    prompt += "s" if int(num_input) > 1 else ""
    prompt += f" and {num_output} {output_type} output"
    prompt += "s" if int(num_output) > 1 else ""
    prompt += "."
    python_code = convert_python_one_line(python_code)

    example_case = {"messages":[{"role": "system", "content": "You are a helpful assistant that generates LangFlow custom component python code based on user descriptions."},{"role": "user", "content": prompt}, {"role": "assistant", "content": python_code}]}

    # Append the example case to the file
    with open(output_file, 'a', encoding='utf-8') as f:
        f.write(json.dumps(example_case) + '\n')
                    
    return len(example_case)
                    

In [28]:
output_file = "python_code_prompt.jsonl"
sample_row = df.iloc[0]  # Get the first row as a Series

convert_python_to_jsonl(
    sample_row["Python Code"], sample_row["Description"], sample_row["Num Input"],
    sample_row["Input type"], sample_row["Num Output"], sample_row["Output type"], output_file
)

1

In [ ]:
output_file = "python_code_prompt.jsonl"
with open(output_file, "w", encoding="utf-8") as f:
    pass

for code, des, num_input, input_type, num_output, output_type in zip(
    df["Python Code"], df["Description"], df["Num Input"],
    df["Input type"], df["Num Output"], df["Output type"]
):
    convert_python_to_jsonl(code, des, num_input, input_type, num_output, output_type, output_file)


2. Code for Fine-tune Python JSONL

In [ ]:
#Uploud to openai
client = OpenAI(api_key = "")
client.files.create(
    file = open("python_code_prompt.jsonl", "rb"),
    purpose="fine-tune"
)

FileObject(id='file-7i34QGeW3QmJhYX9QbVbSv', bytes=76135, created_at=1739858735, filename='python_code_prompt.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [39]:
#Fine-tune the openai model 
client.fine_tuning.jobs.create(
  training_file="file-7i34QGeW3QmJhYX9QbVbSv",
  model="gpt-4o-mini-2024-07-18"
) 

FineTuningJob(id='ftjob-2Ps63ZqGDvheKs9Wr0mGRApE', created_at=1739859105, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-qCGDW63V9bnCi52O10lwRZlm', result_files=[], seed=1101403155, status='validating_files', trained_tokens=None, training_file='file-7i34QGeW3QmJhYX9QbVbSv', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'batch_size': 'auto', 'learning_rate_multiplier': 'auto', 'n_epochs': 'auto'}}})

In [41]:
# Replace with your actual fine-tuning job ID
fine_tune_job_id = "ftjob-2Ps63ZqGDvheKs9Wr0mGRApE"
# 0.23$
# Retrieve job status
job_status = client.fine_tuning.jobs.retrieve(fine_tune_job_id)
print(job_status)

FineTuningJob(id='ftjob-2Ps63ZqGDvheKs9Wr0mGRApE', created_at=1739859105, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::B2BEJt6D', finished_at=1739859521, hyperparameters=Hyperparameters(n_epochs=5, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-qCGDW63V9bnCi52O10lwRZlm', result_files=['file-2fztvUM2L8ETLpS9R3uZRu'], seed=1101403155, status='succeeded', trained_tokens=78165, training_file='file-7i34QGeW3QmJhYX9QbVbSv', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 5, 'batch_size': 1, 'learning_rate_multiplier': 1.8}}})


3. Test Python JSONL 

In [52]:
#Test Python JSONL
from openai import OpenAI

client = OpenAI()

prompt = '''Generate a LangFlow custom component in python code that multiplies 2 numbers. 
The component should have 2 Message inputs and 1 Message output. Here is an example of a Echo function:
"# from langflow.field_typing import Data\nfrom langflow.custom import Component\nfrom langflow.io import MessageTextInput, Output\nfrom langflow.schema import Data\n\n\nclass CustomComponent(Component):\n    display_name = \"Custom Component\"\n    description = \"Use as a template to create your own component.\"\n    documentation: str = \"http://docs.langflow.org/components/custom\"\n    icon = \"code\"\n    name = \"CustomComponent\"\n\n    inputs = [\n        MessageTextInput(\n            name=\"input_value\",\n            display_name=\"Input Value\",\n            info=\"This is a custom component Input\",\n            value=\"Hello, World!\",\n            tool_mode=True,\n        ),\n    ]\n\n    outputs = [\n        Output(display_name=\"Output\", name=\"output\", method=\"build_output\"),\n    ]\n\n    def build_output(self) -> Data:\n        data = Data(value=self.input_value)\n        self.status = data\n        return data\n",                       
'''

response = client.chat.completions.create(
    model="ft:gpt-4o-mini-2024-07-18:personal::B2BEJt6D",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)


You can create a custom component in LangFlow to multiply two numbers by following the structure of the provided example. Below is the code that demonstrates how to implement a `CustomComponent` named `"Multiplier"`. 

Here's the code:

```python
from langflow.field_typing import Data
from langflow.custom import Component
from langflow.io import MessageTextInput, Output
from langflow.schema import Data


class MultiplierComponent(Component):
    display_name = "Multiplier"
    description = "Multiplies two numbers."
    icon = "calculator"
    name = "MultiplierComponent"

    inputs = [
        MessageTextInput(
            name="input_a",
            display_name="Input A",
            info="First number to multiply.",
            value="1",
            tool_mode=True,
        ),
        MessageTextInput(
            name="input_b",
            display_name="Input B",
            info="Second number to multiply.",
            value="1",
            tool_mode=True,
        ),
    ]

 

In [ ]:
#Test Python JSONL
from openai import OpenAI

client = OpenAI()

prompt = '''Generate a LangFlow custom component in python code that extracts spoken content from YouTube videos with DataFrame output options.
The component should have 1 Message inputs  for URL and 1 Message output. Here is an example of a Echo function:
"# from langflow.field_typing import Data\nfrom langflow.custom import Component\nfrom langflow.io import MessageTextInput, Output\nfrom langflow.schema import Data\n\n\nclass CustomComponent(Component):\n    display_name = \"Custom Component\"\n    description = \"Use as a template to create your own component.\"\n    documentation: str = \"http://docs.langflow.org/components/custom\"\n    icon = \"code\"\n    name = \"CustomComponent\"\n\n    inputs = [\n        MessageTextInput(\n            name=\"input_value\",\n            display_name=\"Input Value\",\n            info=\"This is a custom component Input\",\n            value=\"Hello, World!\",\n            tool_mode=True,\n        ),\n    ]\n\n    outputs = [\n        Output(display_name=\"Output\", name=\"output\", method=\"build_output\"),\n    ]\n\n    def build_output(self) -> Data:\n        data = Data(value=self.input_value)\n        self.status = data\n        return data\n
'''

response = client.chat.completions.create(
    model="ft:gpt-4o-mini-2024-07-18:personal::B2BEJt6D",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)


To create a custom component in LangFlow that extracts spoken content from YouTube videos and provides DataFrame output options, we will leverage the existing structure of a custom component. We'll make use of libraries like `youtube_transcript_api` for subtitles and `pandas` for structuring the output as a DataFrame.

Here's how you would implement this:

### CustomComponent.py

```python
from langflow.custom import Component
from langflow.field_typing import Data
from langflow.io import MessageTextInput, Output
from langflow.schema import Data
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter

import pandas as pd

class YouTubeTranscriptsComponent(Component):
    display_name = "YouTube Transcripts"
    description = "Get transcripts from YouTube videos."
    documentation: str = "http://docs.langflow.org/components/custom"
    icon = "youtube"
    name = "YouTubeTranscripts"

    inputs = [
        MessageTextInput(
 

Prepare json data

In [ ]:
#Prepare JSON data to JSONL
df_json = pd.read_excel(r"C:\\Users\\btuud\\Downloads\\Component_generator\\train_langflow_custom_component_data.xlsx")
print(df_json)

json2prompt

In [82]:
def json_to_prompt(component_json, python_code, num_input, input_type, num_output, output_type, output_file):
    python_code = convert_python_one_line(python_code)
    #print(python_code)
    prompt = f"Generate a LangFlow component JSON for the python code that matches: {python_code}. Include {num_input} {input_type} input{'s' if int(num_input) > 1 else ''} and {num_output} {output_type} output{'s' if int(num_output) > 1 else ''}. Leave 'value' field empty."


    example_case = {"messages":[{"role":"system","content":"Create LangFlow component JSON configurations matching Python implementations."},{"role":"user","content":prompt},{"role":"assistant","content":component_json}]}
    # Append the example case to the file
    #print(example_case)
    with open(output_file, 'a', encoding='utf-8') as f:
        f.write(json.dumps(example_case, ensure_ascii=False) + '\n')
    
    return len(example_case)

In [80]:
output_file_json = "json_prompt.jsonl"
with open(output_file_json, "w", encoding="utf-8") as f:
    pass 
sample_row = df_json.iloc[0]  # Get the first row as a Series
#print(convert_python_one_line(sample_row["Python Code"]))
json_to_prompt(
    sample_row["Component JSON"],sample_row["Python Code"], sample_row["Num Input"],
    sample_row["Input type"] , sample_row["Num Output"], sample_row["Output type"],output_file_json 
)

1

In [83]:
output_file_json = "json_prompt.jsonl"

with open(output_file_json, "w", encoding="utf-8") as f:
    pass 

for component_json, python_code, num_input, input_type, num_ouput, out_put_type in zip(
    df_json["Component JSON"],df_json["Python Code"], df_json["Num Input"],
    df_json["Input type"] , df_json["Num Output"], df_json["Output type"] 
):
    json_to_prompt(component_json, python_code, num_input, input_type, num_ouput, out_put_type, output_file_json)

Test openAI

In [ ]:
#Uploud to openai
client = OpenAI(api_key = "")
client.files.create(
    file = open("json_prompt.jsonl", "rb"),
    purpose="fine-tune"
)

FileObject(id='file-8zPGQYuATp51XQ3YT4YkKe', bytes=207204, created_at=1739948248, filename='json_prompt.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [87]:
#Fine-tune the openai model 
client.fine_tuning.jobs.create(
  training_file="file-8zPGQYuATp51XQ3YT4YkKe",
  model="gpt-4o-mini-2024-07-18"
) 

FineTuningJob(id='ftjob-8hXZFz1g20jVHQPGPsLGjoHk', created_at=1739948327, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-qCGDW63V9bnCi52O10lwRZlm', result_files=[], seed=1633221412, status='validating_files', trained_tokens=None, training_file='file-8zPGQYuATp51XQ3YT4YkKe', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'batch_size': 'auto', 'learning_rate_multiplier': 'auto', 'n_epochs': 'auto'}}})

In [91]:
# Replace with your actual fine-tuning job ID
fine_tune_job_id = "ftjob-8hXZFz1g20jVHQPGPsLGjoHk"
# 0.40$
# Retrieve job status
job_status = client.fine_tuning.jobs.retrieve(fine_tune_job_id)
print(job_status)

FineTuningJob(id='ftjob-8hXZFz1g20jVHQPGPsLGjoHk', created_at=1739948327, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=5, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-qCGDW63V9bnCi52O10lwRZlm', result_files=[], seed=1633221412, status='running', trained_tokens=None, training_file='file-8zPGQYuATp51XQ3YT4YkKe', validation_file=None, estimated_finish=1739948649, integrations=[], user_provided_suffix=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 5, 'batch_size': 1, 'learning_rate_multiplier': 1.8}}})


In [93]:
#Test Python JSONL
from openai import OpenAI

client = OpenAI()
#0.01$
prompt = '''Generate a LangFlow component JSON for {"data":{"edges":[],"nodes":[{"data":{"node":{"template":{"_type":"Component","code":{"type":"code","required":true,"placeholder":"","list":false,"show":true,"multiline":true,"value":"","fileTypes":[],"file_path":"","password":false,"name":"code","advanced":true,"dynamic":true,"info":"","load_from_db":false,"title_case":false},"input_a":{"tool_mode":true,"trace_as_input":true,"trace_as_metadata":true,"load_from_db":false,"list":false,"required":false,"placeholder":"","show":true,"name":"input_a","value":"","display_name":"Input A","advanced":false,"input_types":["Message"],"dynamic":false,"info":"First number to add","title_case":false,"type":"str","_input_type":"MessageTextInput"},"input_b":{"tool_mode":true,"trace_as_input":true,"trace_as_metadata":true,"load_from_db":false,"list":false,"required":false,"placeholder":"","show":true,"name":"input_b","value":"","display_name":"Input B","advanced":false,"input_types":["Message"],"dynamic":false,"info":"Second number to add","title_case":false,"type":"str","_input_type":"MessageTextInput"}},"description":"Takes two inputs and returns their sum as a chat message","icon":"calculator","base_classes":["Message"],"display_name":"A + B","documentation":"http://docs.langflow.org/components/custom","custom_fields":{},"output_types":[],"pinned":false,"conditional_paths":[],"frozen":false,"outputs":[{"types":["Message"],"selected":"Message","name":"message","display_name":"Message","method":"build_output","value":"__UNDEFINED__","cache":true}],"field_order":["input_a","input_b"],"beta":false,"legacy":false,"edited":true,"metadata":{},"tool_mode":false,"lf_version":"1.1.1","official":false},"type":"CustomComponent","id":"CustomComponent-6zrlw"},"id":"CustomComponent-6zrlw","position":{"x":0,"y":0},"type":"genericNode"}],"viewport":{"x":1,"y":1,"zoom":1}},"description":"Takes two inputs and returns their sum as a chat message","name":"A + B","id":"CustomComponent-6zrlw","is_component":true,"last_tested_version":"1.1.1"}.
Include 2 Message inputs and 1 Message output. Leave 'value' field empty.
'''

response = client.chat.completions.create(
    model="ft:gpt-4o-mini-2024-07-18:personal::B2YQNexS",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)


{"data":{"edges":[],"nodes":[{"data":{"node":{"template":{"_type":"Component","code":{"type":"code","required":true,"placeholder":"","list":false,"show":true,"multiline":true,"value":"def run(input_a: str, input_b: str) -> str:\n    \"\"\"Takes two inputs and returns their sum as a chat message\"\"\"\n    return str(int(input_a) + int(input_b))\n","fileTypes":[],"file_path":"","password":false,"name":"code","advanced":true,"dynamic":true,"info":"","load_from_db":false,"title_case":false},"input_a":{"tool_mode":true,"trace_as_input":true,"trace_as_metadata":true,"load_from_db":false,"list":false,"required":false,"placeholder":"","show":true,"name":"input_a","value":"","display_name":"Input A","advanced":false,"input_types":["Message"],"dynamic":false,"info":"First number to add","title_case":false,"type":"str","_input_type":"MessageTextInput"},"input_b":{"tool_mode":true,"trace_as_input":true,"trace_as_metadata":true,"load_from_db":false,"list":false,"required":false,"placeholder":"","s